In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

DATA_PATH = 'C:/Users/MJ/Desktop/MyValue/NLP/Model/data/' #데이터경로 설정
print('파일 크기: ')
for file in os.listdir(DATA_PATH):
    if 'txt' in file:
        print(file.ljust(30)+str(round(os.path.getsize(DATA_PATH+ file) / 100000,2))+'MB')

In [ ]:
#트레인 파일 불러오기
train_data = pd.read_csv(DATA_PATH + 'DATASET.txt', header = 0, delimiter = '\t', quoting=3, encoding='cp949')
train_data.head()

In [ ]:
print('학습데이터 전체 개수: {}'.format(len(train_data)))

In [ ]:
#리뷰 전체길이 확인
train_length = train_data['발화'].astype(str).apply(len)
train_length.head()

In [ ]:
#리뷰 통계 정보
print('리뷰 길이 최댓값: {}'.format(np.max(train_length)))
print('리뷰 길이 최솟값: {}'.format(np.min(train_length)))
print('리뷰 길이 평균값: {:.2f}'.format(np.mean(train_length)))
print('리뷰 길이 표준편차: {:.2f}'.format(np.std(train_length)))
print('리뷰 길이 중간값: {}'.format(np.median(train_length)))
print('리뷰 길이 제1사분위: {}'.format(np.percentile(train_length,25)))
print('리뷰 길이 제3사분위: {}'.format(np.percentile(train_length,75)))

In [ ]:
# 문자열 아닌 데이터 모두 제거
train_review = [review for review in train_data['발화'] if type(review) is str]
train_review

In [ ]:
# 한글 폰트 설정(.ttf파일 다운로드 후 실행)
wordcloud = WordCloud(DATA_PATH+'ROKAF Sans Medium.ttf').generate(' '.join(train_review))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
#긍정 1, 부정 0
print('불안 리뷰 갯수: {}'.format(train_data['감정'].value_counts()[0]))
print('슬픔 리뷰 갯수: {}'.format(train_data['감정'].value_counts()[1]))
print('분노 리뷰 갯수: {}'.format(train_data['감정'].value_counts()[2]))
print('기쁨 리뷰 갯수: {}'.format(train_data['감정'].value_counts()[3]))
print('중립 리뷰 갯수: {}'.format(train_data['감정'].value_counts()[4]))

In [ ]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

DATA_PATH = 'C:/Users/MJ/Desktop/MyValue/NLP/Model/data/'
train_data = pd.read_csv(DATA_PATH + 'DATASET.txt', header = 0, delimiter = '\t', quoting=3, encoding='cp949')

train_data['발화'][:5]

In [ ]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
  #함수인자설명
  # review: 전처리할 텍스트
  # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
  # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
  # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

  # 1. 한글 및 공백 제외한 문자 모두 제거
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)
  
  #2. okt 객체를 활용하여 형태소 단어로 나눔
  word_review = okt.morphs(review_text,stem=True)

  if remove_stopwords:
    #3. 불용어 제거(선택)
    word_review = [token for token in word_review if not token in stop_words]
  return word_review

In [ ]:
# 전체 텍스트 전처리
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
okt = Okt()
clean_train_review = []

for review in train_data['발화']:
  # 리뷰가 문자열인 경우만 전처리 진행
  if type(review) == str:
    clean_train_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words= stop_words))
  else:
    clean_train_review.append([]) #str이 아닌 행은 빈칸으로 놔두기

clean_train_review[:4]